In [19]:
from core import *
import warnings
warnings.filterwarnings('ignore')

In [57]:
cell_path1 = './lr/after/datas/all_a/all_a.h5ad' #path for single-cell data1
folder_path1 = './lr/after/datas/all1/'    #folder to save the results
radius1 = 0 # parameter for converting into meta cell

cell_path2 = './lr/after/datas/all_b/all_b.h5ad' #path for single-cell data2
folder_path2 = './lr/after/datas/all2/'  
radius2 = 0

resolution=0.3 #parameter for clustering
method='average' # method we calcuclate the center value 
 
n_pca=10 #pca number
meta_col = 'batch' #celltype col in the single-cell file
epoches1 = 1 #number for embedding
epoches2 = 0 #number for rotation

c1=0.125 # threshold for merging
c2=0.0  # threshold for merging

In [58]:
merge_by_radius(cell_path1,folder_path1,radius1,method,meta_col)


100%|████████████████████████████████████| 800/800 [00:00<00:00, 3318934.92it/s]


In [59]:
merge_by_radius(cell_path2,folder_path2,radius2,method,meta_col)


100%|████████████████████████████████████| 800/800 [00:00<00:00, 1673537.76it/s]


In [60]:
adata1 = sc.read_h5ad(folder_path1+ 'adata.h5ad')
adata2 = sc.read_h5ad(folder_path2+ 'adata.h5ad')

In [65]:
preprocessing_cluster(adata1,N_pcs=n_pca,resolution=resolution)
preprocessing_cluster(adata2,N_pcs=n_pca,resolution=resolution)

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1)
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)

meta_list1 = calculate_cluster_celltype(adata1);
meta_list2 = calculate_cluster_celltype(adata2);

In [68]:
adata1.obs['leiden'] = adata1.obs['celltype']
adata2.obs['leiden'] = adata2.obs['celltype']

In [69]:

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1)
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)

meta_list1 = calculate_cluster_celltype(adata1);
meta_list2 = calculate_cluster_celltype(adata2);

In [70]:
names = np.load('./lr/after/datas/all_a/dataname.npy')
names = np.array(['0','3','1','2'])
names

array(['0', '3', '1', '2'], dtype='<U1')

In [71]:
fathers = np.load('./lr/after/datas/all_a/datalink.npy')
fathers = np.array([-1,0,1,0])
fathers

array([-1,  0,  1,  0])

In [72]:
xys = np.load('./lr/after/datas/all_a/dataxy.npy')
xys = xys[:4]
xys

array([[ 0.0002292302,  0.0018557151],
       [-0.0102608452,  0.0024107302],
       [-0.0101403088,  0.0002014315],
       [-0.0101124933,  0.0022132014]])

In [87]:
folder_path1

'./lr/after/datas/all1/'

In [93]:
names

array(['0', '3', '1', '2'], dtype='<U1')

In [94]:
np.save(folder_path1+"dataname.npy",names)
np.save(folder_path1+"datalink.npy",fathers)
np.save(folder_path1+"dataxy.npy",xys)

In [95]:
np.save(folder_path2+"dataname.npy",names)
np.save(folder_path2+"datalink.npy",fathers)
np.save(folder_path2+"dataxy.npy",xys)

In [96]:
datas1 = sc.read_h5ad('./lr/after/datas/all_a.h5ad')
datas2 = sc.read_h5ad('./lr/after/datas/all_b.h5ad')

datas1.obsm

AxisArrays with keys: meta

In [97]:
datas1

AnnData object with n_obs × n_vars = 800 × 19738
    obs: 'batch'
    obsm: 'meta'

In [98]:
adata1

AnnData object with n_obs × n_vars = 800 × 2000
    obs: 'celltype', 'leiden'
    uns: 'pca', 'neighbors', 'diffmap_evals', 'umap', 'leiden', 'paga', 'leiden_sizes'
    obsm: 'X_pca', 'X_diffmap', 'X_umap', 'meta'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [99]:
temp1 = datas1.obsm['meta']
temp1.index = adata1.to_df().index
temp2 = datas2.obsm['meta']
temp2.index = adata2.to_df().index


In [100]:
adata1.obsm['meta'] = temp1
adata2.obsm['meta'] = temp2

In [101]:
folder_path1

'./lr/after/datas/all1/'

In [105]:
nodes1 = build_hyper_tree_from_folder(folder_path1,False)
nodes2 = build_hyper_tree_from_folder(folder_path2,False)

nodes_merge1 = [];
nodes_merge2 = [];
add_meta(nodes1[0],meta_list1,[])
add_meta(nodes2[0],meta_list2,[])


In [106]:
datas1.obs['leiden'] = adata1.obs['leiden'].tolist()
datas2.obs['leiden'] = adata2.obs['leiden'].tolist()




In [108]:
nodes2

[0_0, 3_3, 1_1, 2_2]

In [109]:
T=tree_alignment(nodes1[0],nodes2[0],1,True,datas1,datas2);
minn = T.run_alignment();
T.show_ans();

start spilting data
View of AnnData object with n_obs × n_vars = 400 × 8426
    obs: 'batch', 'leiden'
    obsm: 'meta'
finish 1
start filtering LRIs with spatial data
reading data
<bound method NDFrame.head of                 cell          x         y  cell_type
0      cell1-0-1-0-0   6.373032  8.933043        1.0
1      cell2-0-1-0-0   1.613726  8.248680        1.0
2      cell3-0-1-0-0   1.373817  2.201218        1.0
3      cell4-0-1-0-0   0.809356  9.415871        1.0
4      cell5-0-1-0-0   2.810129  6.630638        1.0
..               ...        ...       ...        ...
395  cell196-1-1-0-1  11.378252  1.025635        2.0
396  cell197-1-1-0-1   7.743184  2.681785        2.0
397  cell198-1-1-0-1   9.194564  6.090764        2.0
398  cell199-1-1-0-1  12.555637  7.740070        2.0
399  cell200-1-1-0-1  12.855511  7.345174        2.0

[400 rows x 4 columns]>
DataFrame 中不存在名为 '_index' 的列。
Index(['cell', 'x', 'y', 'cell_type'], dtype='object')
st_gene_len----8426
cell_name_list----400
r

In [110]:
ans = T.get_ans()
G=show_graph(ans,nodes1[0],nodes2[0]);
G.show_fig()

{'2': '#eeea5e', '0': '#44d641', '1': '#cc4bc3', '3': '#6b252f', 'empty': '#fff'}


In [12]:
datas1.obs

,batch
cell,
cell1-0-0-0-0,0
cell2-0-0-0-0,0
cell3-0-0-0-0,0
cell4-0-0-0-0,0
cell5-0-0-0-0,0
...,...
cell496-0-1,3
cell497-0-1,3
cell498-0-1,3


In [15]:
t1 = [1+int(i)/100 for i in datas1.obs['leiden'].tolist()]
t2 = [2+int(i)/100 for i in datas2.obs['leiden'].tolist()]

t1

[1.0,
 1.0,
 1.02,
 1.0,
 1.0,
 1.0,
 1.02,
 1.0,
 1.03,
 1.02,
 1.02,
 1.02,
 1.01,
 1.0,
 1.02,
 1.03,
 1.01,
 1.01,
 1.0,
 1.03,
 1.01,
 1.0,
 1.01,
 1.01,
 1.03,
 1.0,
 1.0,
 1.01,
 1.03,
 1.02,
 1.02,
 1.0,
 1.03,
 1.0,
 1.01,
 1.02,
 1.0,
 1.0,
 1.01,
 1.0,
 1.01,
 1.01,
 1.0,
 1.01,
 1.02,
 1.0,
 1.02,
 1.0,
 1.03,
 1.03,
 1.01,
 1.01,
 1.02,
 1.01,
 1.0,
 1.03,
 1.01,
 1.0,
 1.01,
 1.03,
 1.0,
 1.03,
 1.03,
 1.0,
 1.03,
 1.0,
 1.0,
 1.01,
 1.02,
 1.01,
 1.03,
 1.01,
 1.01,
 1.02,
 1.02,
 1.03,
 1.02,
 1.02,
 1.0,
 1.0,
 1.0,
 1.02,
 1.01,
 1.0,
 1.02,
 1.0,
 1.0,
 1.0,
 1.0,
 1.03,
 1.02,
 1.0,
 1.02,
 1.02,
 1.01,
 1.03,
 1.0,
 1.03,
 1.02,
 1.0,
 1.01,
 1.01,
 1.02,
 1.0,
 1.01,
 1.0,
 1.0,
 1.0,
 1.02,
 1.0,
 1.02,
 1.03,
 1.02,
 1.03,
 1.0,
 1.02,
 1.02,
 1.0,
 1.03,
 1.0,
 1.0,
 1.02,
 1.02,
 1.02,
 1.03,
 1.0,
 1.0,
 1.0,
 1.01,
 1.01,
 1.02,
 1.01,
 1.03,
 1.02,
 1.02,
 1.03,
 1.03,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.01,
 1.01,
 1.0,
 1.0,
 1.01,
 1.03,
 1.0,
 1.0,
 1.02,


In [16]:
datas1.obsm['meta']['cell_type']=t1
datas2.obsm['meta']['cell_type']=t2

datas1.obsm['meta'],datas2.obsm['meta'],

(                    x         y  cell_type
 meta_label                                
 0            0.964426  8.021405       1.00
 1            6.955575  6.196758       1.00
 2            4.854014  3.126034       1.02
 3            4.588672  1.858687       1.00
 4            6.527263  3.105629       1.00
 ...               ...       ...        ...
 1995         9.263169  0.555852       1.00
 1996         9.669555  3.791322       1.03
 1997         9.320879  7.972610       1.01
 1998        10.587842  3.240921       1.03
 1999        12.626369  5.229619       1.02
 
 [2000 rows x 3 columns],
                     x         y  cell_type
 meta_label                                
 0           15.234337  4.520728       2.03
 1           16.711325  1.565761       2.02
 2           18.488227  8.491433       2.01
 3           13.203376  7.788988       2.02
 4           18.677738  4.831916       2.02
 ...               ...       ...        ...
 1995        12.334517  8.577428       2.03
 199

In [17]:
tdatas = datas1.concatenate(datas2)
tdatas.obsm['meta']
tdatas.write_h5ad('./lr/after/datas/temp/AN.h5ad')

In [18]:
tdatas.obsm['meta']['cell_type'].unique()

array([1.  , 1.02, 1.03, 1.01, 2.03, 2.02, 2.01, 2.  , 2.04])

In [27]:




n='temp'

'python3 lr/after/CytoTour.py lr/after/datas/lr_database.csv lr/after/datas/pathways.tsv lr/after/datas/{}/split 10000 lr/after/datas/{}/AN.h5ad --out lr/after/datas/{}'.format(n,n,n)



'python3 lr/after/CytoTour.py lr/after/datas/lr_database.csv lr/after/datas/pathways.tsv lr/after/datas/temp/split 10000 lr/after/datas/temp/AN.h5ad --out lr/after/datas/temp'

In [28]:
'python3 lr/after/splith5ad.py lr/after/datas/{}/AN.h5ad lr/after/datas/{}/split lr/after/datas/pathways.tsv 10000'.format(n,n)


'python3 lr/after/splith5ad.py lr/after/datas/temp/AN.h5ad lr/after/datas/temp/split lr/after/datas/pathways.tsv 10000'

In [ ]:
import glob
files = glob.glob('./lr/after/datas/temp/*')
files

In [ ]:
rate,anslist,ans = run_alignment(nodes1,nodes2,folder_path1,folder_path2,meta_list1,meta_list2);


In [ ]:
def lr_s(n):
    
    a1= 'python3 lr/after/splith5ad.py lr/after/datas/{}/AN.h5ad lr/after/datas/{} lr/after/datas/pathways.tsv 1000'.format(n,n)
    a2 = 'python3 lr/after/CytoTour.py lr/after/datas/lr_database.csv lr/after/datas/pathways.tsv lr/after/datas/{} 1000 lr/after/datas/{}/AN.h5ad --out lr/after/datas/{}'.format(n,n,n)
    return a1,a2 
a1,a2 = lr_s('temp')
os.system(a1)
os.system(a2)
df = pd.read_pickle('./lr/after/datas/temp/cci_result_1.0_2.0.pkl')
df['lr_score']['score'].mean()

In [ ]:
df = pd.read_pickle('./lr/after/datas/temp/cci_result_1.0_2.0.pkl')
df['lr_score']['score'].mean()